<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
OnlyInfKernel
</b></h1> 

In [1]:
print("hello wrld")

hello wrld


### **ℹ️INFO**
* First, we would like to thank our participants for sharing their excellent baselines.
    * [Two‑Branch Human‑Activity‑Recognition Pipeline (IMU + Thermopile/TOF  + SE‑CNN + BiLSTM + Attentio](https://www.kaggle.com/code/vonmainstein/imu-tof)

### **ℹ️IMU+THM/TOF Great Related works(Published)**
* Thanks for sharing your implementation of the new feature.
    * LB.75 [CMI25 | IMU+THM/TOF |TF BiLSTM+GRU+Attention|LB.75](https://www.kaggle.com/code/hideyukizushi/cmi25-imu-thm-tof-tf-bilstm-gru-attention-lb-75)
    * LB.76 [IMU Signal Processing Optimization](https://www.kaggle.com/code/rktqwe/lb-0-76-imu-thm-tof-tf-bilstm-gru-attention)
    * LB.77 [Gravity Component Removal from Accelerometer Data](https://www.kaggle.com/code/rktqwe/lb-0-77-linear-accel-tf-bilstm-gru-attention)
    * LB.78 [New Feature: Angular Velocity from Quaternion Derivatives](https://www.kaggle.com/code/nksusth/lb-0-78-quaternions-tf-bilstm-gru-attention)

---

### **ℹ️MyUpdate**
* In previous notebooks that used "IMU+TOF" as features, the validation strategy was TTS, which raised concerns about its robustness.
For this reason, we will release an implementation that simply blends five models and a model with a high ValidationScore.

```
predictions = []
for model in models:
    idx = int(model.predict(pad_input, verbose=0).argmax(1)[0])
    predictions.append(idx)

idx = max(set(predictions), key=predictions.count)
return str(gesture_classes[idx])
```

* The validationScore for the local training  model included in this notebook is
    * [ModelWeight](https://www.kaggle.com/datasets/hideyukizushi/20250627-cmi-b-102-b-105)

```
0.891134700273056
0.8912659261884439
0.8914825129445727
0.8915471835009202
0.8922128108549205
```


<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
Model Detail
</b></h1> 

<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
InfPipeline
</b></h1> 

### 》》》**Importing the necessary Libraries**

In [2]:
import os, json, joblib, numpy as np, pandas as pd
from pathlib import Path
import warnings 
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.utils import Sequence, to_categorical, pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, Conv1D, BatchNormalization, Activation, add, MaxPooling1D, Dropout,
    Bidirectional, LSTM, GlobalAveragePooling1D, Dense, Multiply, Reshape,
    Lambda, Concatenate, GRU, GaussianNoise
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import tensorflow as tf
import polars as pl
from sklearn.model_selection import StratifiedGroupKFold
from scipy.spatial.transform import Rotation as R
import re



2025-07-07 14:16:21.303337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751897781.507915      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751897781.564989      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### 》》》**Fix Seed**

In [3]:
import random
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
seed_everything(seed=42)

### 》》》**Configuration**

In [4]:
# (Competition metric will only be imported when TRAINing)
TRAIN = False# ← set to True when you want to train
RAW_DIR = Path("/kaggle/input/cmi-detect-behavior-with-sensor-data")
PRETRAINED_DIR = Path("/kaggle/input/lb-0-78-quaternions-tf-bilstm-gru-attention")  # used when TRAIN=False
EXPORT_DIR = Path("./")                                    # artefacts will be saved here
BATCH_SIZE = 64
PAD_PERCENTILE = 95
LR_INIT = 5e-4
WD = 3e-3
MIXUP_ALPHA = 0.4
EPOCHS = 160
PATIENCE = 40

ifFreqWidth = True
ifFreqEntropy = True
ifEquivalentToUpright = True

print("▶ imports ready · tensorflow", tf.__version__)

▶ imports ready · tensorflow 2.18.0


### 》》》**Utility Functions**

In [5]:
#Tensor Manipulations
def time_sum(x):
    return K.sum(x, axis=1)

def squeeze_last_axis(x):
    return tf.squeeze(x, axis=-1)

def expand_last_axis(x):
    return tf.expand_dims(x, axis=-1)

def se_block(x, reduction=8):
    ch = x.shape[-1]
    se = GlobalAveragePooling1D()(x)
    se = Dense(ch // reduction, activation='relu')(se)
    se = Dense(ch, activation='sigmoid')(se)
    se = Reshape((1, ch))(se)
    return Multiply()([x, se])

# Residual CNN Block with SE
def residual_se_cnn_block(x, filters, kernel_size, pool_size=2, drop=0.3, wd=1e-4):
    shortcut = x
    for _ in range(2):
        x = Conv1D(filters, kernel_size, padding='same', use_bias=False,
                   kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    x = se_block(x)
    if shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, 1, padding='same', use_bias=False,
                          kernel_regularizer=l2(wd))(shortcut)
        shortcut = BatchNormalization()(shortcut)
    x = add([x, shortcut])
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size)(x)
    x = Dropout(drop)(x)
    return x

def attention_layer(inputs):
    score = Dense(1, activation='tanh')(inputs)
    score = Lambda(squeeze_last_axis)(score)
    weights = Activation('softmax')(score)
    weights = Lambda(expand_last_axis)(weights)
    context = Multiply()([inputs, weights])
    context = Lambda(time_sum)(context)
    return context

### 》》》**Data Helpers**

In [6]:
# Normalizes and cleans the time series sequence. 

def preprocess_sequence(df_seq: pd.DataFrame, feature_cols: list[str], scaler: StandardScaler):
    mat = df_seq[feature_cols].ffill().bfill().fillna(0).values
    return scaler.transform(mat).astype('float32')

# MixUp the data argumentation in order to regularize the neural network. 

class MixupGenerator(Sequence):
    def __init__(self, X, y, batch_size, alpha=0.2):
        self.X, self.y = X, y
        self.batch = batch_size
        self.alpha = alpha
        self.indices = np.arange(len(X))
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch))
    def __getitem__(self, i):
        idx = self.indices[i*self.batch:(i+1)*self.batch]
        Xb, yb = self.X[idx], self.y[idx]
        lam = np.random.beta(self.alpha, self.alpha)
        perm = np.random.permutation(len(Xb))
        X_mix = lam * Xb + (1-lam) * Xb[perm]
        y_mix = lam * yb + (1-lam) * yb[perm]
        return X_mix, y_mix
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [7]:
def remove_gravity_from_acc(acc_data, rot_data):

    if isinstance(acc_data, pd.DataFrame):
        acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    else:
        acc_values = acc_data

    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    
    gravity_world = np.array([0, 0, 9.81])

    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :] 
            continue

        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
             
    return linear_accel

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200): # Assuming 200Hz sampling rate
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))

    for i in range(num_samples - 1):
        q_t = quat_values[i]
        q_t_plus_dt = quat_values[i+1]

        if np.all(np.isnan(q_t)) or np.all(np.isclose(q_t, 0)) or \
           np.all(np.isnan(q_t_plus_dt)) or np.all(np.isclose(q_t_plus_dt, 0)):
            continue

        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)

            # Calculate the relative rotation
            delta_rot = rot_t.inv() * rot_t_plus_dt
            
            # Convert delta rotation to angular velocity vector
            # The rotation vector (Euler axis * angle) scaled by 1/dt
            # is a good approximation for small delta_rot
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            # If quaternion is invalid, angular velocity remains zero
            pass
            
    return angular_vel

## *v20: added FFT and freq analysis to feature engineering*


In [8]:
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt

# 假设 fs 是采样率
fs = 100  # 设置采样率为 100Hz，当然根据你的数据来调整

def FFTforLinearAccel(linear_accel):
    """
    对加速度的 x, y, z 分量分别进行傅里叶变换，并返回各个分量的 FFT 结果。
    
    参数:
    - linear_accel: 三维加速度数据，形状为 (n_samples, 3)，每一列对应 x, y, z 方向的加速度。
    
    返回:
    - fft_results: 包含 x, y, z 分量 FFT 结果的字典。
    """
    accelX = linear_accel[:, 0]  # x方向的加速度
    accelY = linear_accel[:, 1]  # y方向的加速度
    accelZ = linear_accel[:, 2]  # z方向的加速度
    
    fft_resultX = np.fft.fft(accelX)  # 对加速度数据做傅里叶变换
    freqsX = np.fft.fftfreq(len(accelX), d=1/fs)  # 计算对应的频率轴

    fft_resultY = np.fft.fft(accelY)  # y方向的傅里叶变换
    freqsY = np.fft.fftfreq(len(accelY), d=1/fs)

    fft_resultZ = np.fft.fft(accelZ)  # z方向的傅里叶变换
    freqsZ = np.fft.fftfreq(len(accelZ), d=1/fs)

    fft_results = {
        'x': (fft_resultX, freqsX),
        'y': (fft_resultY, freqsY),
        'z': (fft_resultZ, freqsZ)
    }
    return fft_results


def freqEntropyFromLinearAccelFFT(fft_results):
    """
    计算从傅里叶变换结果得到的频域熵。
    
    参数:
    - fft_results: 包含傅里叶变换结果和频率轴的字典。
    
    返回:
    - spectral_entropy: 频域熵值。
    """
    # 计算幅度
    amplitude_x = np.abs(fft_results['x'][0])
    amplitude_y = np.abs(fft_results['y'][0])
    amplitude_z = np.abs(fft_results['z'][0])

    # 归一化幅度
    amplitude_x_norm = amplitude_x / np.sum(amplitude_x)
    amplitude_y_norm = amplitude_y / np.sum(amplitude_y)
    amplitude_z_norm = amplitude_z / np.sum(amplitude_z)
    
    # 计算频域熵
    spectral_entropy_x = entropy(amplitude_x_norm)
    spectral_entropy_y = entropy(amplitude_y_norm)
    spectral_entropy_z = entropy(amplitude_z_norm)
    '''
    print(f"Spectral Entropy for X: {spectral_entropy_x}")
    print(f"Spectral Entropy for Y: {spectral_entropy_y}")
    print(f"Spectral Entropy for Z: {spectral_entropy_z}")
    '''
    
    
    return spectral_entropy_x, spectral_entropy_y, spectral_entropy_z
'''

# 测试：生成一组 dummy 数据并进行验证
np.random.seed(42)  # 保证结果可复现
dummy_data = np.random.randn(1000, 3)  # 生成 1000 个随机数据点，模拟加速度（x, y, z）
# 对加速度数据进行傅里叶变换并计算频域熵
fft_results = FFTforLinearAccel(dummy_data)
spectral_entropy = freqEntropyFromLinearAccelFFT(fft_results)


'''




'\n\n# 测试：生成一组 dummy 数据并进行验证\nnp.random.seed(42)  # 保证结果可复现\ndummy_data = np.random.randn(1000, 3)  # 生成 1000 个随机数据点，模拟加速度（x, y, z）\n# 对加速度数据进行傅里叶变换并计算频域熵\nfft_results = FFTforLinearAccel(dummy_data)\nspectral_entropy = freqEntropyFromLinearAccelFFT(fft_results)\n\n\n'

## v47 exploring non-linear dynamics and modifications to FFT 


In [9]:
def calculate_angular_distance(rot_data):
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)

    for i in range(num_samples - 1):
        q1 = quat_values[i]
        q2 = quat_values[i+1]

        if np.all(np.isnan(q1)) or np.all(np.isclose(q1, 0)) or \
           np.all(np.isnan(q2)) or np.all(np.isclose(q2, 0)):
            angular_dist[i] = 0 # Or np.nan, depending on desired behavior
            continue
        try:
            # Conversion of quaternions to Rotation objects
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)

            # Calculating angular distance: 2 * arccos(|real(p * q*)|)
            # where p* is the conjugate quaternion of q
            # In scipy.spatial.transform.Rotation, r1.inv() * r2 gives the relative rotation.
            # The angle of this relative rotation is the angular distance.
            relative_rotation = r1.inv() * r2
            
            # The angle of the rotation vector corresponds to the angular distance
            # The norm of the rotation vector is the angle in radians
            angle = np.linalg.norm(relative_rotation.as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            angular_dist[i] = 0 # In case of invalid quaternions
            pass
            
    return angular_dist


In [10]:
import re
from scipy.spatial.transform import Rotation as R
import numpy as np

def equivalentToSittingStraight(linear_accel, orientation = "seated straight"):
    """
    将加速度数据转换为坐直（sitting straight）姿势下的加速度。
    
    参数：
    - linear_accel: 三维加速度数据（如经过重力去除后的平动加速度）。
    - orientation: 当前的姿势信息（如 "seated lean", "seated straight", "lie on side" 等）。
    
    返回：
    - 转换为坐正姿势后的加速度数据。
    """
    # 检查 "seated lean" 姿势
    if re.search(r'\bseated\s*lean\b', orientation, re.IGNORECASE):
        rotation = R.from_euler('xyz', [0, -30, 0], degrees=True)  # 负的 Pitch
    # 检查 "seated straight" 姿势
    elif re.search(r'\bseated\s*straight\b', orientation, re.IGNORECASE):
        rotation = R.from_euler('xyz', [0, 0, 0], degrees=True)  # 没有旋转
    # 检查 "lie on side" 姿势
    elif re.search(r'\blie\s*on\s*side\b', orientation, re.IGNORECASE):
        rotation = R.from_euler('xyz', [90, 90, 0], degrees=True)  # 90° Roll 和 90° Pitch
    # 检查 "lie on back" 姿势
    elif re.search(r'\blie\s*on\s*back\b', orientation, re.IGNORECASE):
        rotation = R.from_euler('xyz', [0, 90, 0], degrees=True)  # 90° Pitch
    else:
        # 如果姿势没有匹配，返回原始加速度
        rotation = R.from_euler('xyz', [0, 0, 0], degrees=True)  # 默认无旋转
    
    # 获取旋转矩阵的逆
    inverseRotation = rotation.inv().as_matrix()

    # 确保 linear_accel 是三维向量（例如形状为 (3,) 或 (n, 3)）
    # 如果 linear_accel 是一个 1D 数组，转换为 (3, ) 的 3D 向量
    if linear_accel.ndim == 1:
        linear_accel = linear_accel.reshape(1, 3)  # 将其转为 1 x 3 数组

    # 使用逆旋转矩阵对加速度进行转换
    adjusted_accel = inverseRotation @ linear_accel.T  # 矩阵乘法，确保形状匹配

    return adjusted_accel.T  # 返回转换后的加速度数据

'''
# 例子
linear_accel = np.array([0.1, 0.2, 0.3])  # 三维加速度向量
orientation = 'seated lean'

adjusted_accel = equivalentToSittingStraight(linear_accel, orientation)
print(f"Adjusted Linear Acceleration: {adjusted_accel}")

'''


'\n# 例子\nlinear_accel = np.array([0.1, 0.2, 0.3])  # 三维加速度向量\norientation = \'seated lean\'\n\nadjusted_accel = equivalentToSittingStraight(linear_accel, orientation)\nprint(f"Adjusted Linear Acceleration: {adjusted_accel}")\n\n'

### 》》》**Model Definition - Two Branch Architecture**

In [11]:
def build_two_branch_model(pad_len, imu_dim, tof_dim, n_classes, wd=1e-4):
    inp = Input(shape=(pad_len, imu_dim+tof_dim))
    imu = Lambda(lambda t: t[:, :, :imu_dim])(inp)
    tof = Lambda(lambda t: t[:, :, imu_dim:])(inp)

    # IMU deep branch
    x1 = residual_se_cnn_block(imu, 64, 3, drop=0.1, wd=wd)
    x1 = residual_se_cnn_block(x1, 128, 5, drop=0.1, wd=wd)

    # TOF/Thermal lighter branch
    x2 = Conv1D(64, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(tof)
    x2 = BatchNormalization()(x2); x2 = Activation('relu')(x2)
    x2 = MaxPooling1D(2)(x2); x2 = Dropout(0.2)(x2)
    x2 = Conv1D(128, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x2)
    x2 = BatchNormalization()(x2); x2 = Activation('relu')(x2)
    x2 = MaxPooling1D(2)(x2); x2 = Dropout(0.2)(x2)

    merged = Concatenate()([x1, x2])

    xa = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xb = Bidirectional(GRU(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xc = GaussianNoise(0.09)(merged)
    xc = Dense(16, activation='elu')(xc)
    
    x = Concatenate()([xa, xb, xc])
    x = Dropout(0.4)(x)
    x = attention_layer(x)

    for units, drop in [(256, 0.5), (128, 0.3)]:
        x = Dense(units, use_bias=False, kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x); x = Activation('relu')(x)
        x = Dropout(drop)(x)

    out = Dense(n_classes, activation='softmax', kernel_regularizer=l2(wd))(x)
    return Model(inp, out)

tmp_model = build_two_branch_model(127,7,325,18)

I0000 00:00:1751897794.805754      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


### 》》》**Training / Inference Pipeline**

In [12]:
custom_objs = {
    'time_sum': time_sum,
    'squeeze_last_axis': squeeze_last_axis,
    'expand_last_axis': expand_last_axis,
    'se_block': se_block,
    'residual_se_cnn_block': residual_se_cnn_block,
    'attention_layer': attention_layer,
}

print("hello world custom objects")

hello world custom objects


In [13]:

if TRAIN:
    print("▶ TRAIN MODE – loading dataset …")
    df = pd.read_csv(RAW_DIR / "train.csv")

    train_dem_df = pd.read_csv(RAW_DIR / "train_demographics.csv")
    df_for_groups = pd.merge(df.copy(), train_dem_df, on='subject', how='left')

    le = LabelEncoder()
    df['gesture_int'] = le.fit_transform(df['gesture'])
    np.save(EXPORT_DIR / "gesture_classes.npy", le.classes_)
    gesture_classes = le.classes_

    print("  Calculating base engineered IMU features (magnitude, angle)...")
    df['acc_mag'] = np.sqrt(df['acc_x']**2 + df['acc_y']**2 + df['acc_z']**2)
    df['rot_angle'] = 2 * np.arccos(df['rot_w'].clip(-1, 1))
    
    print("  Calculating engineered IMU derivatives (jerk, angular velocity) for original acc_mag...")
    df['acc_mag_jerk'] = df.groupby('sequence_id')['acc_mag'].diff().fillna(0)
    df['rot_angle_vel'] = df.groupby('sequence_id')['rot_angle'].diff().fillna(0)

    print("  Removing gravity and calculating linear acceleration features...")

    linear_accel_list = []

    
    freq_entropy_x_list = []
    freq_entropy_y_list = []
    freq_entropy_z_list = []

    spectral_flatness_x_list = []
    spectral_flatness_y_list = []
    spectral_flatness_z_list = []

    for _, group in df.groupby('sequence_id'):
        acc_data_group = group[['acc_x', 'acc_y', 'acc_z']]
        rot_data_group = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
        pose = group['orientation'].iloc[0]
        linear_accel_group = remove_gravity_from_acc(acc_data_group, rot_data_group)
        #print("preTarget")
        if ifEquivalentToUpright:
            linear_accel_group = equivalentToSittingStraight (linear_accel_group, pose)
        #print("postTarget")
        linear_accel_list.append(pd.DataFrame(linear_accel_group, columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group.index))

        fft_results = FFTforLinearAccel(linear_accel_group)

        if ifFreqWidth:

            amplitude_x = np.abs(fft_results['x'][0])
            amplitude_y = np.abs(fft_results['y'][0])
            amplitude_z = np.abs(fft_results['z'][0])
    
            
            print("Freq entropy from lindear acceleration")
    
            print("Spectral flatness")
            spectral_flatness_x = np.exp(np.mean(np.log(amplitude_x + 1e-8))) / np.mean(amplitude_x)
            spectral_flatness_y = np.exp(np.mean(np.log(amplitude_y + 1e-8))) / np.mean(amplitude_y)
            spectral_flatness_z = np.exp(np.mean(np.log(amplitude_z + 1e-8))) / np.mean(amplitude_z)
    
            
            
            spectral_flatness_x_list.append(spectral_flatness_x)
            spectral_flatness_y_list.append(spectral_flatness_y)
            spectral_flatness_z_list.append(spectral_flatness_z)

        if ifFreqEntropy:
            freq_entropy_x, freq_entropy_y, freq_entropy_z = freqEntropyFromLinearAccelFFT(fft_results)
        

            freq_entropy_x_list.append(freq_entropy_x)
            freq_entropy_y_list.append(freq_entropy_y)
            freq_entropy_z_list.append(freq_entropy_z)
        
    

        #print("FFT freq analysis of linear acceleration")
        
        


    print(freq_entropy_x_list)
    df_linear_accel = pd.concat(linear_accel_list)
    df = pd.concat([df, df_linear_accel], axis=1)
    if ifFreqWidth:
         df['spectral_flatness_x'] = pd.concat([pd.Series([x]) for x in spectral_flatness_x_list], axis=0).reset_index(drop=True)
         df['spectral_flatness_y'] = pd.concat([pd.Series([x]) for x in spectral_flatness_y_list], axis=0).reset_index(drop=True)
         df['spectral_flatness_z'] = pd.concat([pd.Series([x]) for x in spectral_flatness_z_list], axis=0).reset_index(drop=True)
    if ifFreqEntropy:
        df['freq_entropy_x'] = pd.concat([pd.Series(x) for x in freq_entropy_x_list], axis=0).reset_index(drop=True)
        df['freq_entropy_y'] = pd.concat([pd.Series(x) for x in freq_entropy_y_list], axis=0).reset_index(drop=True)
        df['freq_entropy_z'] = pd.concat([pd.Series(x) for x in freq_entropy_z_list], axis=0).reset_index(drop=True)

        
    

    # Convert each float in the lists to a pandas Series before concatenating
    
    df['linear_acc_mag'] = np.sqrt(df['linear_acc_x']**2 + df['linear_acc_y']**2 + df['linear_acc_z']**2)
    df['linear_acc_mag_jerk'] = df.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0)

    print("  Calculating angular velocity from quaternion derivatives...")
    angular_vel_list = []
    for _, group in df.groupby('sequence_id'):
        rot_data_group = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
        angular_vel_group = calculate_angular_velocity_from_quat(rot_data_group)
        angular_vel_list.append(pd.DataFrame(angular_vel_group, columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=group.index))
    
    df_angular_vel = pd.concat(angular_vel_list)
    df = pd.concat([df, df_angular_vel], axis=1)

    print("  Calculating angular distance between successive quaternions...")
    angular_distance_list = []
    for _, group in df.groupby('sequence_id'):
        rot_data_group = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
        angular_dist_group = calculate_angular_distance(rot_data_group)
        angular_distance_list.append(pd.DataFrame(angular_dist_group, columns=['angular_distance'], index=group.index))
    
    df_angular_distance = pd.concat(angular_distance_list)
    df = pd.concat([df, df_angular_distance], axis=1)

    meta_cols = { } # This was an empty dict in your provided code, keeping it as is.

    imu_cols_base = ['linear_acc_x', 'linear_acc_y', 'linear_acc_z']
    imu_cols_base.extend([c for c in df.columns if c.startswith('rot_') and c not in ['rot_angle', 'rot_angle_vel']])
    
    imu_engineered_features = [
        'acc_mag', 'rot_angle',
        'acc_mag_jerk', 'rot_angle_vel',
        'linear_acc_mag', 'linear_acc_mag_jerk',
        'angular_vel_x', 'angular_vel_y', 'angular_vel_z', # Existing new features
        'angular_distance' # Added new feature  
    ]
    if ifFreqEntropy:
        imu_engineered_features += ['freq_entropy_x', 'freq_entropy_y', 'freq_entropy_z']
    if ifFreqWidth:
        imu_engineered_features += ['spectral_flatness_x', 'spectral_flatness_y', 'spectral_flatness_z']

    imu_cols = imu_cols_base + imu_engineered_features
    imu_cols = list(dict.fromkeys(imu_cols)) # Для удаления дубликатов

    thm_cols_original = [c for c in df.columns if c.startswith('thm_')]
    
    tof_aggregated_cols_template = []
    for i in range(1, 6):
        tof_aggregated_cols_template.extend([f'tof_{i}_mean', f'tof_{i}_std', f'tof_{i}_min', f'tof_{i}_max'])

    final_feature_cols = imu_cols + thm_cols_original + tof_aggregated_cols_template
    imu_dim_final = len(imu_cols)
    tof_thm_aggregated_dim_final = len(thm_cols_original) + len(tof_aggregated_cols_template)
    
    print(f"  IMU (incl. engineered & derivatives) {imu_dim_final} | THM + Aggregated TOF {tof_thm_aggregated_dim_final} | total {len(final_feature_cols)} features")
    np.save(EXPORT_DIR / "feature_cols.npy", np.array(final_feature_cols))

    print("  Building sequences with aggregated TOF and preparing data for scaler...")
    seq_gp = df.groupby('sequence_id') 
    
    all_steps_for_scaler_list = []
    X_list_unscaled, y_list_int_for_stratify, lens = [], [], [] 

    for seq_id, seq_df_orig in seq_gp:
        seq_df = seq_df_orig.copy()

        for i in range(1, 6):
            pixel_cols_tof = [f"tof_{i}_v{p}" for p in range(64)]
            tof_sensor_data = seq_df[pixel_cols_tof].replace(-1, np.nan)
            seq_df[f'tof_{i}_mean'] = tof_sensor_data.mean(axis=1)
            seq_df[f'tof_{i}_std']  = tof_sensor_data.std(axis=1)
            seq_df[f'tof_{i}_min']  = tof_sensor_data.min(axis=1)
            seq_df[f'tof_{i}_max']  = tof_sensor_data.max(axis=1)
        
        mat_unscaled = seq_df[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32')
        
        all_steps_for_scaler_list.append(mat_unscaled)
        X_list_unscaled.append(mat_unscaled)
        y_list_int_for_stratify.append(seq_df['gesture_int'].iloc[0])
        lens.append(len(mat_unscaled))

    print("  Fitting StandardScaler...")
    all_steps_concatenated = np.concatenate(all_steps_for_scaler_list, axis=0)
    scaler = StandardScaler().fit(all_steps_concatenated)
    joblib.dump(scaler, EXPORT_DIR / "scaler.pkl")
    del all_steps_for_scaler_list, all_steps_concatenated

    print("  Scaling and padding sequences...")
    X_scaled_list = [scaler.transform(x_seq) for x_seq in X_list_unscaled]
    del X_list_unscaled

    pad_len = int(np.percentile(lens, PAD_PERCENTILE))
    np.save(EXPORT_DIR / "sequence_maxlen.npy", pad_len)
    
    X = pad_sequences(X_scaled_list, maxlen=pad_len, padding='post', truncating='post', dtype='float32')
    del X_scaled_list
    
    y_int_for_stratify = np.array(y_list_int_for_stratify)
    y = to_categorical(y_int_for_stratify, num_classes=len(le.classes_))

    print("  Splitting data and preparing for training...")
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=82, stratify=y_int_for_stratify)

    cw_vals = compute_class_weight('balanced', classes=np.arange(len(le.classes_)), y=y_int_for_stratify)
    class_weight = dict(enumerate(cw_vals))

    model = build_two_branch_model(pad_len, imu_dim_final, tof_thm_aggregated_dim_final, len(le.classes_), wd=WD)
    
    steps = len(X_tr) // BATCH_SIZE
    lr_sched = tf.keras.optimizers.schedules.CosineDecayRestarts(5e-4, first_decay_steps=15 * steps) 
    
    model.compile(optimizer=Adam(lr_sched),
                  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                  metrics=['accuracy'])

    train_gen = MixupGenerator(X_tr, y_tr, batch_size=BATCH_SIZE, alpha=MIXUP_ALPHA)
    cb = EarlyStopping(patience=PATIENCE, restore_best_weights=True, verbose=1, monitor='val_accuracy', mode='max')
    
    print("  Starting model training...")
    model.fit(train_gen, epochs=EPOCHS, validation_data=(X_val, y_val),
              class_weight=class_weight, callbacks=[cb], verbose=1)

    model.save(EXPORT_DIR / "gesture_two_branch_mixup.h5")
    print("✔ Training done – artefacts saved in", EXPORT_DIR)

    from cmi_2025_metric_copy_for_import import CompetitionMetric
    preds_val = model.predict(X_val).argmax(1)
    true_val_int  = y_val.argmax(1)
    
    h_f1 = CompetitionMetric().calculate_hierarchical_f1(
        pd.DataFrame({'gesture': le.classes_[true_val_int]}),
        pd.DataFrame({'gesture': le.classes_[preds_val]}))
    print("Hold‑out H‑F1 =", round(h_f1, 4))

     # 保存特征配置
    '''

    feature_config = {
        "ifFreqWidth": ifFreqWidth,
        "ifFreqEntropy": ifFreqEntropy,
        "ifEquivalentToUpright": ifEquivalentToUpright
    }
    with open(EXPORT_DIR / "feature_config.json", "w") as f:
        json.dump(feature_config, f)
    
    print("✔ Feature configuration saved to feature_config.json")
    
    '''
    
else:
    '''
    # 从导出目录读取特征配置
    with open(PRETRAINED_DIR / "feature_config.json") as f:
        feature_config = json.load(f)
    
    # 强制覆盖当前全局变量
    ifFreqWidth = feature_config["ifFreqWidth"]
    ifFreqEntropy = feature_config["ifFreqEntropy"]
    ifEquivalentToUpright = feature_config["ifEquivalentToUpright"]

    print(f"Loaded feature configuration: {feature_config}")

    '''
    
    print("▶ INFERENCE MODE – loading artefacts from", PRETRAINED_DIR)
    final_feature_cols = np.load(PRETRAINED_DIR / "feature_cols.npy", allow_pickle=True).tolist()
    pad_len        = int(np.load(PRETRAINED_DIR / "sequence_maxlen.npy"))
    scaler         = joblib.load(PRETRAINED_DIR / "scaler.pkl")
    gesture_classes = np.load(PRETRAINED_DIR / "gesture_classes.npy", allow_pickle=True)

    # Re-calculate imu_dim_final based on the actual features that will be used
    # Убедитесь, что 'angular_distance' учитывается здесь при инференсе
    imu_features_in_final_cols = [c for c in final_feature_cols if any(c.startswith(prefix) for prefix in ['linear_acc_', 'acc_', 'rot_', 'angular_vel_', 'angular_distance'])]
    imu_dim_final = len(imu_features_in_final_cols)

    tof_thm_aggregated_dim_final = len(final_feature_cols) - imu_dim_final

   
    
    model = load_model(PRETRAINED_DIR / "gesture_two_branch_mixup.h5",
                       compile=False, custom_objects=custom_objs)
    print("  Model, scaler, feature_cols, pad_len loaded – ready for evaluation")

   

▶ INFERENCE MODE – loading artefacts from /kaggle/input/lb-0-78-quaternions-tf-bilstm-gru-attention
  Model, scaler, feature_cols, pad_len loaded – ready for evaluation


<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
Blending Model
</b></h1> 

### 》》》**LoadBlendingModels**

In [14]:
models = []

model = load_model("/kaggle/input/lb-0-78-quaternions-tf-bilstm-gru-attention/gesture_two_branch_mixup.h5",compile=False, custom_objects=custom_objs)
models.append(model)

'''
model = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.8914825129445727_.h5",compile=False, custom_objects=custom_objs)
models.append(model)
model = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.8912659261884439_.h5",compile=False, custom_objects=custom_objs)
models.append(model)
model = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.891134700273056_.h5",compile=False, custom_objects=custom_objs)
models.append(model)
model = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.8915471835009202_.h5",compile=False, custom_objects=custom_objs)
models.append(model)
model = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.8922128108549205_.h5",compile=False, custom_objects=custom_objs)
models.append(model)
'''




'\nmodel = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.8914825129445727_.h5",compile=False, custom_objects=custom_objs)\nmodels.append(model)\nmodel = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.8912659261884439_.h5",compile=False, custom_objects=custom_objs)\nmodels.append(model)\nmodel = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.891134700273056_.h5",compile=False, custom_objects=custom_objs)\nmodels.append(model)\nmodel = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.8915471835009202_.h5",compile=False, custom_objects=custom_objs)\nmodels.append(model)\nmodel = load_model("/kaggle/input/20250627-cmi-b-102-b-105/0.8922128108549205_.h5",compile=False, custom_objects=custom_objs)\nmodels.append(model)\n'

### 》》》**Predict**

In [15]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    df_seq = sequence.to_pandas()

    df_seq['acc_mag'] = np.sqrt(df_seq['acc_x']**2 + df_seq['acc_y']**2 + df_seq['acc_z']**2)
    df_seq['rot_angle'] = 2 * np.arccos(df_seq['rot_w'].clip(-1, 1))
    df_seq['acc_mag_jerk'] = df_seq['acc_mag'].diff().fillna(0)
    df_seq['rot_angle_vel'] = df_seq['rot_angle'].diff().fillna(0)

    

    acc_cols_for_gravity_removal = ['acc_x', 'acc_y', 'acc_z']
    rot_cols_for_gravity_removal = ['rot_x', 'rot_y', 'rot_z', 'rot_w']

    if not all(col in df_seq.columns for col in acc_cols_for_gravity_removal + rot_cols_for_gravity_removal):
        print(f"Warning: Missing raw acc/rot columns for gravity removal in predict for sequence. Using raw acc as linear.")
        df_seq['linear_acc_x'] = df_seq.get('acc_x', 0)
        df_seq['linear_acc_y'] = df_seq.get('acc_y', 0)
        df_seq['linear_acc_z'] = df_seq.get('acc_z', 0)
    else:
        acc_data_seq = df_seq[acc_cols_for_gravity_removal]
        rot_data_seq = df_seq[rot_cols_for_gravity_removal]
        linear_accel_seq_arr = remove_gravity_from_acc(acc_data_seq, rot_data_seq)
        
        df_seq['linear_acc_x'] = linear_accel_seq_arr[:, 0]
        df_seq['linear_acc_y'] = linear_accel_seq_arr[:, 1]
        df_seq['linear_acc_z'] = linear_accel_seq_arr[:, 2]


    
    if ifEquivalentToUpright:
        print (f" ifEquivalentToUpright is {ifEquivalentToUpright }")
        if "orientation" in df_seq.columns:
            pose = df_seq["orientation"].iloc[0]
        else:
            pose = "seated straight"
        
        linear_accel_seq_arr = equivalentToSittingStraight (linear_accel_seq_arr, pose)
        
        
        
    df_seq['linear_acc_mag'] = np.sqrt(df_seq['linear_acc_x']**2 + df_seq['linear_acc_y']**2 + df_seq['linear_acc_z']**2)
    df_seq['linear_acc_mag_jerk'] = df_seq['linear_acc_mag'].diff().fillna(0)

    # FFT frequency analysis

    # 更新频率特征
    fft_results = FFTforLinearAccel(df_seq[['linear_acc_x', 'linear_acc_y', 'linear_acc_z']].values)
    amplitude_x = np.abs(fft_results["x"])
    amplitude_y = np.abs(fft_results['y'])
    amplitude_z = np.abs(fft_results["z"])

    spectral_entropy_x, spectral_entropy_y, spectral_entropy_z = freqEntropyFromLinearAccelFFT(fft_results)

    # 更新频率特征
    if ifFreqEntropy:
        df_seq['freq_entropy_x'] = spectral_entropy_x
        df_seq['freq_entropy_y'] = spectral_entropy_y
        df_seq['freq_entropy_z'] = spectral_entropy_z

    # 计算频谱平坦度

    if ifFreqWidth:
        spectral_flatness_x = np.exp(np.mean(np.log(amplitude_x + 1e-8))) / np.mean(amplitude_x)
        spectral_flatness_y = np.exp(np.mean(np.log(amplitude_y + 1e-8))) / np.mean(amplitude_y)
        spectral_flatness_z = np.exp(np.mean(np.log(amplitude_z + 1e-8))) / np.mean(amplitude_z)
    
        # 更新频谱平坦度特征
        
        df_seq['spectral_flatness_x'] = spectral_flatness_x
        df_seq['spectral_flatness_y'] = spectral_flatness_y
        df_seq['spectral_flatness_z'] = spectral_flatness_z

    # 确保所有其他需要的特征也在 df_seq 中
    df_seq_final_features = pd.DataFrame(index=df_seq.index)
    for col_name in final_feature_cols:
        if col_name in df_seq.columns:
            df_seq_final_features[col_name] = df_seq[col_name]
        else:
            print(f"CRITICAL ERROR IN PREDICT: Feature '{col_name}' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.")
            df_seq_final_features[col_name] = 0
    
    # Calculate angular velocity from quaternions in predict function
    if all(col in df_seq.columns for col in rot_cols_for_gravity_removal):
        angular_vel_seq_arr = calculate_angular_velocity_from_quat(df_seq[rot_cols_for_gravity_removal])
        df_seq['angular_vel_x'] = angular_vel_seq_arr[:, 0]
        df_seq['angular_vel_y'] = angular_vel_seq_arr[:, 1]
        df_seq['angular_vel_z'] = angular_vel_seq_arr[:, 2]
    else:
        print(f"Warning: Missing quaternion columns for angular velocity calculation in predict. Filling with 0.")
        df_seq['angular_vel_x'] = 0
        df_seq['angular_vel_y'] = 0
        df_seq['angular_vel_z'] = 0

    # Calculate angular distance from quaternions in predict function
    if all(col in df_seq.columns for col in rot_cols_for_gravity_removal):
        angular_dist_seq_arr = calculate_angular_distance(df_seq[rot_cols_for_gravity_removal])
        df_seq['angular_distance'] = angular_dist_seq_arr
    else:
        print(f"Warning: Missing quaternion columns for angular distance calculation in predict. Filling with 0.")
        df_seq['angular_distance'] = 0


    for i in range(1, 6): 
        pixel_cols_tof = [f"tof_{i}_v{p}" for p in range(64)]
        if not all(col in df_seq.columns for col in pixel_cols_tof):
            print(f"Warning: Missing some TOF pixel columns for tof_{i} in predict. Filling aggregates with 0.")
            df_seq[f'tof_{i}_mean'] = 0
            df_seq[f'tof_{i}_std']  = 0
            df_seq[f'tof_{i}_min']  = 0
            df_seq[f'tof_{i}_max']  = 0
            continue

        tof_sensor_data = df_seq[pixel_cols_tof].replace(-1, np.nan)
        df_seq[f'tof_{i}_mean'] = tof_sensor_data.mean(axis=1)
        df_seq[f'tof_{i}_std']  = tof_sensor_data.std(axis=1)
        df_seq[f'tof_{i}_min']  = tof_sensor_data.min(axis=1)
        df_seq[f'tof_{i}_max']  = tof_sensor_data.max(axis=1)
        
    if 'tof_range_across_sensors' in final_feature_cols:
        tof_mean_cols_for_contrast = [f'tof_{i}_mean' for i in range(1, 6) if f'tof_{i}_mean' in df_seq.columns]
        thm_cols_for_contrast = [f'thm_{i}' for i in range(1, 6) if f'thm_{i}' in df_seq.columns]

        if tof_mean_cols_for_contrast:
            tof_values_for_contrast = df_seq[tof_mean_cols_for_contrast]
            df_seq['tof_range_across_sensors'] = tof_values_for_contrast.max(axis=1) - tof_values_for_contrast.min(axis=1)
            df_seq['tof_std_across_sensors'] = tof_values_for_contrast.std(axis=1)
        else:
            df_seq['tof_range_across_sensors'] = 0
            df_seq['tof_std_across_sensors'] = 0

        if thm_cols_for_contrast:
            thm_values_for_contrast = df_seq[thm_cols_for_contrast]
            df_seq['thm_range_across_sensors'] = thm_values_for_contrast.max(axis=1) - thm_values_for_contrast.min(axis=1)
            df_seq['thm_std_across_sensors'] = thm_values_for_contrast.std(axis=1)
        else:
            df_seq['thm_range_across_sensors'] = 0
            df_seq['thm_std_across_sensors'] = 0






    
    df_seq_final_features = pd.DataFrame(index=df_seq.index)
    for col_name in final_feature_cols:
        if col_name in df_seq.columns:
            df_seq_final_features[col_name] = df_seq[col_name]
        else:
            print(f"CRITICAL ERROR IN PREDICT: Feature '{col_name}' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.")
            df_seq_final_features[col_name] = 0 
            
    mat_unscaled = df_seq_final_features.ffill().bfill().fillna(0).values.astype('float32')
    
    mat_scaled = scaler.transform(mat_unscaled)
    
    pad_input = pad_sequences([mat_scaled], maxlen=pad_len, padding='post', truncating='post', dtype='float32')
    
    # ---------------------------------------------- #
    # Blending Models
    # ---------------------------------------------- #

    '''
    no blending
    predictions = []
    for model in models:
        idx = int(model.predict(pad_input, verbose=0).argmax(1)[0])
        predictions.append(idx)
    
    idx = max(set(predictions), key=predictions.count)
    return str(gesture_classes[idx])
    '''
    prediction = model.predict(pad_input, verbose=1)
    idx = int(prediction.argmax(1)[0])
    return str(gesture_classes[idx])

    

### 》》》**Submit Inference server**

In [16]:
import kaggle_evaluation.cmi_inference_server
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )

 ifEquivalentToUpright is True
CRITICAL ERROR IN PREDICT: Feature 'angular_vel_x' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'angular_vel_y' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'angular_vel_z' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'angular_distance' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'tof_1_mean' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'tof_1_std' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
C

2025-07-07 14:16:39.125102: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
I0000 00:00:1751897800.085554      58 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
 ifEquivalentToUpright is True
CRITICAL ERROR IN PREDICT: Feature 'angular_vel_x' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'angular_vel_y' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'angular_vel_z' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'angular_distance' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'tof_1_mean' expected by model (from final_feature_cols) was NOT generated in df_seq. Filling with 0. THIS IS LIKELY A BUG.
CRITICAL ERROR IN PREDICT: Feature 'tof_1_std' expected by model (from final_feature_cols) was NOT generated in df_seq. Fil

In [17]:
dummyList = [
        'acc_mag', 'rot_angle',
        'acc_mag_jerk', 'rot_angle_vel',
        'linear_acc_mag', 'linear_acc_mag_jerk',
        'angular_vel_x', 'angular_vel_y', 'angular_vel_z', # Existing new features
        'angular_distance' # Added new feature
        
        
        
    ]
print(len (dummyList))

10


**测试和评估**